In [454]:
%matplotlib inline
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
from os import listdir
from os.path import isfile, join
from __future__ import print_function
from numpy import median, diff

In [626]:
sample_rate = 22050*8
class SongFile:
    def load_new(self):
        data, _ = librosa.load(self.music_file, sr=sample_rate)
        _, self.beat_frames = librosa.beat.beat_track(y=data, sr=sample_rate)
        self.beat_times = librosa.frames_to_time(self.beat_frames, sr=sample_rate)

        seconds = len(data) / sample_rate
        bpm = get_beats(self.beat_times, self.beat_frames)[0][1]
        bps = bpm / 60
        # take 24 samples for each beat (need 3rds, 8ths)
        num_samples = int(bps * seconds * 24) - 1 # -1 to avoid out of bounds for now
        beat_time = (1. / bps) / 24
        sample_times = [beat_times[0] + (beat_time * i) for i in range(num_samples)]
        self.music_samples = [data[int(time * sample_rate)] for time in sample_times]
    
    def __init__(self, pack, name, extension, force_new):
        self.pack = pack
        self.name = name
        self.extension = extension
        self.music_file = 'StepMania/Songs/{0}/{1}/{1}.{2}'.format(pack, name, extension)
        self.stepfile = 'StepMania/Songs/{0}/{1}/{1}.ssc'.format(pack, name)
        
        key = '{0}~{1}'.format(pack, name)
        if force_new or (not '{0}_beat_frames.csv'.format(key) in listdir('data')):
            print ('Calculating beats for {0}'.format(key))
            self.load_new()
                        
            print ('Saving calculated beats for {0}'.format(key))
            pd.DataFrame(self.beat_times).to_csv('data/{0}_beat_times.csv'.format(key), index=False)
            pd.DataFrame(self.beat_frames).to_csv('data/{0}_beat_frames.csv'.format(key), index=False)
            pd.DataFrame(self.music_samples).to_csv('data/{0}_music_samples.csv'.format(key), index=False)
        else:
            print ('Loading beats from save for {0}'.format(key))
            self.beat_frames = pd.read_csv('data/{0}_beat_frames.csv'.format(key)).values.flatten()
            self.beat_times = pd.read_csv('data/{0}_beat_times.csv'.format(key)).values.flatten()
            self.music_samples = pd.read_csv('data/{0}_music_samples.csv'.format(key)).values.flatten()

In [627]:
song_data = {}

In [628]:
def load_song(pack, title, force_new = False):
    folder_path = 'StepMania/Songs/{0}/{1}'.format(pack, title)
    song = next(file for file in listdir(folder_path) if file.endswith('.ogg') or file.endswith('.mp3'))
    extension = song.split('.')[1]
    key = '{0}~{1}'.format(pack, title)
    song_data[key] = SongFile(pack, title, extension, force_new)

In [636]:
songs = [song for song in listdir('StepMania/Songs/In The Groove')]
songs.remove('.DS_Store')
for song in songs:
    load_song('In The Groove', song, True)
    
load_song('_test', 'A', True)
load_song('_test', 'B', True)
load_song('_test', 'C', True)

Calculating beats for In The Groove~Anubis
Saving calculated beats for In The Groove~Anubis
Calculating beats for In The Groove~Bend Your Mind
Saving calculated beats for In The Groove~Bend Your Mind
Calculating beats for In The Groove~Boogie Down
Saving calculated beats for In The Groove~Boogie Down
Calculating beats for In The Groove~Bouff
Saving calculated beats for In The Groove~Bouff
Calculating beats for In The Groove~Bubble Dancer
Saving calculated beats for In The Groove~Bubble Dancer
Calculating beats for In The Groove~Changes
Saving calculated beats for In The Groove~Changes
Calculating beats for In The Groove~Charlene
Saving calculated beats for In The Groove~Charlene
Calculating beats for In The Groove~Crazy
Saving calculated beats for In The Groove~Crazy
Calculating beats for In The Groove~Da Roots
Saving calculated beats for In The Groove~Da Roots
Calculating beats for In The Groove~Dawn
Saving calculated beats for In The Groove~Dawn
Calculating beats for In The Groove~De

IndexError: index 19053386 is out of bounds for axis 0 with size 19053312

In [639]:
def get_bpm(file):
    with open(file, "r") as ins:
        for line in ins:
            if line.startswith('#BPMS:'):
                result = re.search('#BPMS:(.*);', line)
                bpm_string = result.group(1)
                if len(bpm_string.split(',')) == 1:
                    return float(bpm_string.split('=')[1])
                return 0
    return 0

In [640]:
songs = [song for song in listdir('StepMania/Songs/In The Groove')]
songs.remove('.DS_Store')
song_bpms = {}
for song in songs:
    song_bpms['In The Groove~{0}'.format(song)] = get_bpm('StepMania/Songs/In The Groove/{0}/{0}.sm'.format(song))
    
song_bpms['_test~A'] = 181.685
song_bpms['_test~B'] = 140
song_bpms['_test~C'] = 180

In [648]:
errors = []
for key in song_data:
    song = song_data[key]
    real_beat = song_bpms[key]
    if real_beat != 0:
        prediced_beat = get_beats(song.beat_times, song.beat_frames)[0][1]
        for i in range (1,4):
            if abs((prediced_beat * (i + 1) / (i)) - real_beat) < abs(prediced_beat - real_beat):
                #prediced_beat *= (i + 1) / (i)
        print ('{0} - Real: {2:.3f}, Error: {1:.3f}'.format('', prediced_beat - real_beat, real_beat))
        errors.append(abs(prediced_beat - real_beat))
print (['{0:.3f}'.format(error) for error in errors])

IndentationError: expected an indented block (<ipython-input-648-fad87fc14a44>, line 10)

In [415]:
def get_beats(beat_times, beat_frames):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    sorted_changes = sorted(changes)
    median = sorted_changes[int(len(changes) / 2)]

    changes_counted = [abs(change - median) < 2 for change in changes]
    time_changes_counted = list(itertools.compress(changes_time, changes_counted))
    average = sum(time_changes_counted) / len(time_changes_counted)
    
    time_differences = []
    earliest_proper_beat = 1
    for i in range(1, len(beat_times) - 1):
        if changes_counted[i] & changes_counted[i - 1]:
            earliest_proper_beat = i
            break
            
    last_proper_beat = len(beat_times) -2
    for i in range(1, len(beat_times) - 1):
        if changes_counted[len(beat_times) - i - 1] & changes_counted[len(beat_times) - i - 2]:
            last_proper_beat = len(beat_times) - i - 1
            break
    
    time_differences = []
    buffer = 5
    for i in range(20):
        start_beat = earliest_proper_beat + buffer * i
        if changes_counted[start_beat] & changes_counted[start_beat - 1]:
            for j in range(20):
                end_beat = last_proper_beat - buffer * j
                if changes_counted[end_beat] & changes_counted[end_beat - 1]:
                    time_differences.append(beat_times[end_beat] - beat_times[start_beat])
        
    # get num beats, round, and make new average
    new_averages = [time_difference / round(time_difference / average) for time_difference in time_differences]
    #print (new_averages)
    new_averages.sort()
    num_averages = len(new_averages)
    #new_average = sum(new_averages[5:num_averages - 5]) / (num_averages - 10)
    new_average = new_averages[int(num_averages/2)]
    return [(0, 60./new_average)]
    
def get_time_string(times):
    time_strings = []
    for time in times:
        if time[0] != 0:
            time_strings.append(',')
        time_strings.append('{:}={:.3f}'.format(time[0], time[1]))
    return ''.join(time_strings)

def test_get_beats(beat_times, beat_frames):
    a = get_beats(beat_times, beat_frames)
    a.append((500, 1))
    time = beat_times[0]
    beat_times_mock = [time]
    for i in range(len(a) - 1):
        for j in range(a[i+1][0] - a[i][0]):
            time += 60/a[i][1]
            beat_times_mock.append(time)

    beat_times_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_pd['REAL'] = True
    beat_times_pd
    beat_times_mock_pd = pd.DataFrame(beat_times, columns=['VALUE'])
    beat_times_mock_pd['REAL'] = False
    both = beat_times_pd.append(beat_times_mock_pd)
    both['BEAT']=both.index
    linestyles = ["--", "-"]*390
    fig, (ax) = plt.subplots(1,1, figsize=(20,7))
    graph = sb.pointplot(x='BEAT', y='VALUE', hue='REAL', linestyles=linestyles, data=both, ax=ax)
    
#test_get_beats(beat_times)

In [309]:
def write_song_header(output_stepfile, info_map):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    header_info = {
        'VERSION': 0.82,
        'TITLE': info_map['song_name'],
        'MUSIC': '{0}.mp3'.format(info_map['song_name']),
        'OFFSET': -0.090,
        'SAMPLESTART': info_map['sample_start'],
        'SAMPLELENGTH': info_map['sample_length']
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, info_map):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': 'Kommisar',
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Beginner',
        'METER': 1,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': info_map['bpm']
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)
        
def write_notes(output_stepfile, info_map):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(80):
        print ("0101\n0001\n0101\n0001\n,", file=output_stepfile)
    print ("0000;", file=output_stepfile)

In [329]:
def get_info_map(song):
    info_map = {}
    info_map['song_name'] = song.name
    info_map['song_format'] = song.extension
    times = get_beats(song.beat_times, song.beat_frames)
    bpm_string = get_time_string(times)
    info_map['bpm'] = bpm_string
    tempo = 60./times[0][1]
    info_map['sample_start'] = 16 * tempo
    info_map['sample_length'] = 32 * tempo
    return info_map

In [330]:
def step_song(song):
    output_stepfile=open(song.stepfile, 'w')
    info_map = get_info_map(song)
    write_song_header(output_stepfile, info_map)
    write_step_header(output_stepfile, info_map)
    write_notes(output_stepfile, info_map)
    output_stepfile.close()

In [333]:
step_song(A)
step_song(B)
step_song(C)